In [26]:
import numpy as np
import pandas as pd
import re
import os

reduced = os.path.exists("data/csv/books_data_reduced.csv") and os.path.exists("data/csv/Books_rating_reduced.csv")

if reduced:
    rating_full = pd.read_csv("data/csv/Books_rating_reduced.csv")
    data_full = pd.read_csv("data/csv/books_data_reduced.csv")
else:
    rating_full = pd.read_csv("data/csv/Books_rating.csv")
    data_full = pd.read_csv("data/csv/books_data.csv")

reduced

True

In [27]:
if not reduced:
    rating = rating_full[:len(rating_full) // 5]
    data = data_full[:len(data_full) // 2]

In [28]:
print(rating.columns)
print(data.columns)

Index(['Id', 'Title', 'Price', 'User_id', 'profileName', 'review/helpfulness',
       'review/score', 'review/time', 'review/summary', 'review/text'],
      dtype='object')
Index(['Title', 'description', 'authors', 'image', 'previewLink', 'publisher',
       'publishedDate', 'infoLink', 'categories', 'ratingsCount'],
      dtype='object')


In [29]:
rating.head(2)

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...


In [30]:
data.head(2)

,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN


In [31]:
print(f'Total Ids: {rating.Id.count()}')
print(f'Total Unique Ids: {rating.Id.nunique()}')
print(f'Total User Ids: {rating.User_id.count()}')
print(f'Total Unique User Ids: {rating.User_id.nunique()}')

Total Ids: 600000
Total Unique Ids: 42732
Total User Ids: 482379
Total Unique User Ids: 304934


In [32]:
print(f'Total Title: {rating.Title.count()}')
print(f'Total Unique Titles: {rating.Title.nunique()}')
print(f'Total Data Titles: {data.Title.count()}')
print(f'Total Unique Data Titles: {data.Title.nunique()}')

Total Title: 599993
Total Unique Titles: 42097
Total Data Titles: 106201
Total Unique Data Titles: 106201


In [33]:
books_reviews = pd.merge(rating,data, on = 'Title')
print(books_reviews.columns)
print(books_reviews.shape)

Index(['Id', 'Title', 'Price', 'User_id', 'profileName', 'review/helpfulness',
       'review/score', 'review/time', 'review/summary', 'review/text',
       'description', 'authors', 'image', 'previewLink', 'publisher',
       'publishedDate', 'infoLink', 'categories', 'ratingsCount'],
      dtype='object')
(600000, 19)


## Q1: Título, autores y editoriales de los libros de categoría "Computers" entre 2000 y 2023 que contengan 'distributed' en su título.

In [34]:

year_regex = re.compile('[^\d]*(\d{4})[^\d]*')
def extract_year(x):
    if x:
        result = year_regex.search(x)
        return int(result.group(1)) if result else None
    return None

data_sanitized = data.dropna(subset=['Title', 'authors', 'categories', 'publishedDate'])
data_sanitized = data_sanitized[['Title', 'authors', 'publisher', 'categories', 'publishedDate']].copy()
data_sanitized['publishedYear'] = data_sanitized['publishedDate'].apply(extract_year)
data_sanitized['publishedDecade'] = data_sanitized['publishedYear'] // 10 * 10

data_filtered = data_sanitized.copy()
data_filtered = data_filtered[data_filtered['categories'] == "['Computers']"]
data_filtered = data_filtered[(data_filtered['publishedYear'] >= 2000) & (data_filtered['publishedYear'] <= 2023)]
data_filtered = data_filtered[data_filtered['Title'].str.contains('distributed', case=False)]
data_filtered[['Title', 'authors', 'publisher', 'publishedYear']]

,Title,authors,publisher,publishedYear
7710,Enterprise Linux at Work: How to Build 10 Dist...,['Stephen Asbury'],John Wiley & Sons Incorporated,2000.0
19672,Open Source Network Administration (Prentice H...,['James M. Kretchmar'],Prentice Hall Professional,2004.0
20607,Distributed Systems Architecture: A Middleware...,"['Arno Puder', 'Kay Römer', 'Frank Pilhofer']",Elsevier,2011.0
23643,IBM&reg; WebSphere&reg; Application Server for...,['James W. Cortada'],MIT Press,2019.0
32110,Holographic Reduced Representation: Distribute...,"['Ignacio Rojas', 'Gonzalo Joya', 'Andreu Cata...",Springer Nature,2021.0
62808,Distributed .NET Programming in VB .NET,['Tom Barnaby'],Apress,2002.0
65765,Distributed Com Application Development Using ...,['Jim Maloney'],Prentice Hall,2000.0
75684,Malware: Fighting Malicious Code (Prentice Hal...,"['Ed Skoudis', 'Lenny Zeltser']",Prentice Hall Professional,2004.0
89152,Inside Distributed COM (Mps),"['Intermational Conference Ifip Tcs 200', 'Int...",Springer Science & Business Media,2000.0
97388,Replication Techniques in Distributed Systems ...,"['Abdelsalam A. Helal', 'Abdelsalam A. Heddaya...",Springer Science & Business Media,2006.0


## Q2: Autores que tengan títulos publicados en al menos 10 décadas distintas

In [35]:
## Autores que tengan títulos publicados en al menos 10 décadas distintas
authors_expanded = data_sanitized.explode('authors')
author_unique_decades = authors_expanded.groupby('authors')['publishedDecade'].nunique().reset_index(name='num_decades')
authors_with_at_least_ten_decades = author_unique_decades[author_unique_decades['num_decades'] >= 10]
authors_with_at_least_ten_decades

,authors,num_decades
18537,['Erle Stanley Gardner'],10
19580,['Francis Parkman'],10
54110,['Rudyard Kipling'],11
62371,['Washington Irving'],10
63580,['William Shakespeare'],12


## Q3: Títulos y autores de los libros publicados en la década del 90' con al menos 500 calificaciones

In [36]:
## Títulos publicados en la década del 90'
data_published_in_90s = data_sanitized[data_sanitized['publishedDecade'] == 1990]
print(f'Total 90 Data Titles: {data_published_in_90s.Title.count()}')
data_published_in_90s.head(3)

## Títulos publicados en la década del 90' con 500 calificaciones o más
rating_published_in_90s = rating.merge(data_published_in_90s, on='Title', how='inner')

# Normalizamos el titulo pasando a minusculas y removimendo espacios a izquierda y derecha
#obsolete: rating_published_date_90['Title'] = rating_published_date_90['Title'].str.lower().str.strip()

rating_published_in_90s_count_by_title = rating_published_in_90s.groupby('Title').size().reset_index(name="amountOfReviews")
rating_published_in_90s_count_by_title_more_than_500_reviews = rating_published_in_90s_count_by_title.loc[rating_published_in_90s_count_by_title['amountOfReviews'] >= 500]
rating_published_in_90s_count_by_title_more_than_500_reviews

# Join con tabla de libros para obtener autores
#obsolete: reduced_data = data[['Title', "authors"]].drop_duplicates()
reduced_data = data_sanitized[['Title', "authors"]]
#obsolete: reduced_data['Title'] = reduced_data['Title'].str.lower().str.strip()
rating_published_in_90s_count_by_title_more_than_500_reviews_with_authors = rating_published_in_90s_count_by_title_more_than_500_reviews.merge(reduced_data, on="Title", how="inner")
#obsolete: rating_published_date_90_count_by_title_more_than_500_reviews_with_authors.drop_duplicates()
rating_published_in_90s_count_by_title_more_than_500_reviews_with_authors

Total 90 Data Titles: 17692


,Title,amountOfReviews,authors
0,A Tree Grows in Brooklyn,1552,['Betty Smith']
1,Black Notice (Random House Large Print),654,['Patricia Daniels Cornwell']
2,Foundation and Empire,609,['D. G. Leahy']
3,I am Legend,794,['Richard Matheson']
4,Jane Eyre (Everyman's Classics),1523,['Charlotte Brontë']
5,Jane Eyre (New Windmill),1523,['Charlotte Brontë']
6,Memoirs of a Geisha (Signed),2693,['Arthur Golden']
7,One Hundred Years of Solitude,907,['Gabriel García Márquez']
8,Pride and Prejudice,2041,['Jane Austen']
9,Red Tent,1736,['Anita Diamant']


## Q4: Top 10 libros con mejor promedio de calificación entre aquellos publicados en la década del 90’ con al menos 500 calificaciones


In [37]:
rating_date_90_at_least_500 = rating[['Title', 'review/score']].merge(
    rating_published_in_90s_count_by_title_more_than_500_reviews['Title'], on='Title', how='inner')
rating_date_90_at_least_500_mean_rating = rating_date_90_at_least_500.groupby(['Title']).mean().sort_values('review/score', ascending=False)
top_10_books = rating_date_90_at_least_500_mean_rating.head(10)
top_10_books

,review/score
Title,
A Tree Grows in Brooklyn,4.705541
Pride and Prejudice,4.528662
Jane Eyre (Everyman's Classics),4.490479
Jane Eyre (New Windmill),4.490479
The Hitchhiker's Guide to the Galaxy,4.457694
Memoirs of a Geisha (Signed),4.386929
Slaughterhouse-Five : Or the Children's Crusade,4.324000
Foundation and Empire,4.279146
One Hundred Years of Solitude,4.239250


## Q5: Título de libros de la categoría "Ficción" cuyo sentimiento de reseña promedio esté en el percentil 90 más alto

In [38]:
# Título de libros de la categoría "Ficción" cuyo sentimiento de reseña promedio sea esté en el percentil 90 más alto
from textblob import TextBlob

def analizar_sentimiento(texto):
    # TextBlob devuelve valores de polaridad de sentimiento en el rango de -1 a 1
    if isinstance(texto, str):
        blob = TextBlob(texto)
        sentimiento = blob.sentiment.polarity
        return sentimiento
    else:
        return None  # Si el texto no es una cadena válida, devolver None

# Sanitizamos el dataset para quedarnos con registros con categorías válidas
#obsolete:data_category_sanitized = data.dropna(subset=['categories'])

# Approach para quedarnos con registros relacionados a fiction
data_category_fiction = data_sanitized[data_sanitized['categories'].str.contains('fiction')]

# Generamos una versión reducida de reseñas que nos sirva para joinear
rating_reduced = rating[['Title', 'review/text']]

# Joineamos los libros de la categoría de Ficción con sus reseñas
data_category_fiction_merged_with_ratings = data_category_fiction.merge(rating_reduced, on='Title', how='inner')

# Calculamos el sentimiento sobre los registros de reseñas
data_category_fiction_merged_with_ratings['sentimentPolarity'] = data_category_fiction_merged_with_ratings['review/text'].apply(analizar_sentimiento)

# Agrupamos por título y calculamos el promedio de la sentimientPolarity

data_category_gruped_mean_sentiment_polarity = data_category_fiction_merged_with_ratings.groupby('Title')['sentimentPolarity'].mean().reset_index()

# Ordenamos los registros por polaridad de sentimiento en orden descendente
#obsolete: data_category_fiction_sorted = data_category_gruped_mean_sentiment_polarity.sort_values(by='sentimentPolarity', ascending=False)

# Calculamos el percentil 90
percentile_90 = data_category_gruped_mean_sentiment_polarity['sentimentPolarity'].quantile(0.9)

# Nos quedamos con los libros que esten en el percentil 90
data_category_fiction_percentile_90 = data_category_gruped_mean_sentiment_polarity[data_category_gruped_mean_sentiment_polarity['sentimentPolarity'] >= percentile_90]

data_category_fiction_percentile_90

,Title,sentimentPolarity
8,A Cache of Jewels and Other Collective Nouns (...,0.503874
31,ABC,0.437500
34,ABCs of the Bible,0.554167
35,ACROSS AMERICA ON AN EMIGRANT TRAIN,0.422041
44,Adventures in Odyssey: The Complete Collection,0.475000
...,...,...
906,Welcome to the Sea of Sand,0.596587
914,When Daddy Prays,0.452083
915,When Dragons' Hearts Were Good,0.488208
916,When We Meet Again (Children of the Promise),0.456710


In [39]:
x = [data_filtered[['Title', 'authors', 'publisher']], authors_with_at_least_ten_decades[['authors']], rating_published_in_90s_count_by_title_more_than_500_reviews_with_authors[['Title', 'authors']], top_10_books, data_category_fiction_percentile_90]

for i, dataset in enumerate(x):
    dataset.to_csv(f"data/kaggle_results/query{i+1}_reduced.csv", index=False, header=False)

In [40]:
top_10_books.to_csv(f"data/kaggle_results/query{4}_reduced.csv", header=False)